In [ ]:
from Data_Loader import load_data
from Train import trainCox_nnet

import torch
import numpy as np
import pandas as pd
from torch.utils.data.dataloader import DataLoader
dtype = torch.FloatTensor

Import the data

In [ ]:
from Data_extraction_lung import data_norm_df_lung, output_df_lung

data_norm_df_lung= data_norm_df_lung.reset_index(drop=True)
output_df_lung = output_df_lung.reset_index(drop=True)

In [ ]:
from Data_extraction_pancreas import data_norm_df_pancreas, output_df_pancreas
data_norm_df_pancreas= data_norm_df_pancreas.reset_index(drop=True)
output_df_pancreas = output_df_pancreas.reset_index(drop=True)

In [ ]:
data = pd.concat([data_norm_df_lung,output_df_lung], axis=1)
x, ytime, yevent, age = load_data(data, dtype)

In [ ]:
data_pancreas = pd.concat([data_norm_df_pancreas,output_df_pancreas], axis=1)
x_p, ytime_p, yevent_p, age_p = load_data(data_pancreas, dtype)

Plot the vital status

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns 
plt.figure(figsize=(8,6))
sns.countplot(x='vital_status', data=data)
ax = plt.gca()
plt.xticks(fontsize = 12)
plt.yticks(fontsize=12)
plt.xlabel('Vital status', fontfamily='times new roman', fontsize = 20)
plt.ylabel('Counts', fontfamily='times new roman', fontsize =20)
plt.title('Vital status distribution for lung cancer',fontfamily='times new roman', fontsize = 20)

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns 
plt.figure(figsize=(8,6))
sns.countplot(x='vital_status', data=data_pancreas)
ax = plt.gca()
plt.xticks(fontsize = 12)
plt.yticks(fontsize=12)
plt.xlabel('Vital status', fontfamily='times new roman', fontsize = 20)
plt.ylabel('Counts', fontfamily='times new roman', fontsize =20)
plt.title('Vital status distribution for pancreatic cancer',fontfamily='times new roman', fontsize = 20)

Plot the survival time distribution

In [ ]:
x_df=pd.DataFrame(x, columns = np.arange(len(x[0,:])))
data_cph = pd.concat([x_df,pd.DataFrame({'age': age[:,0], 'yevent': yevent[:,0], 'ytime': ytime[:,0]})], axis=1)
essai = data_cph.drop(data[data_cph.yevent==0].index)
essai2 = data_cph.drop(data_cph[data_cph.yevent==1].index)

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns 

plt.figure(figsize=(10,6))
# plt.subplot(211)
sns.distplot(essai['ytime'], kde=False, label='dead patients')
ax = plt.gca()
plt.xticks(fontsize = 12)
plt.yticks(fontsize=12)
plt.xlabel('Survival time [days]', fontfamily='times new roman', fontsize = 20)
plt.ylabel('Counts', fontfamily='times new roman', fontsize =20)
plt.title('Survival time distribution for lung cancer',fontfamily='times new roman', fontsize = 20)

# plt.legend()
# plt.subplot(212)
# sns.distplot(essai2['ytime'], kde=False, label='patients alive')
# plt.xlabel('days_to_last_follow_up')
# plt.legend()


In [ ]:
x_df_p=pd.DataFrame(x_p, columns = np.arange(len(x_p[0,:])))
data_cph_p = pd.concat([x_df_p,pd.DataFrame({'age_p': age_p[:,0], 'yevent_p': yevent_p[:,0], 'ytime_p': ytime_p[:,0]})], axis=1)
essai_p = data_cph_p.drop(data_pancreas[data_cph_p.yevent_p==0].index)
essai2_p = data_cph_p.drop(data_cph_p[data_cph_p.yevent_p==1].index)

In [ ]:
plt.figure(figsize=(10,6))
# plt.subplot(211)
sns.distplot(essai_p['ytime_p'], kde=False, label='dead patients')
ax = plt.gca()
plt.xticks(fontsize = 12)
plt.yticks(fontsize=12)
plt.xlabel('Survival time [days]', fontfamily='times new roman', fontsize = 20)
plt.ylabel('Counts', fontfamily='times new roman', fontsize =20)
plt.title('Survival time distribution for pancreatic cancer',fontfamily='times new roman', fontsize = 20)

Plot the age distribution

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns 



plt.figure(figsize=(10,6))
plt.subplot(211)
sns.distplot(essai['age']/365, kde=False, label='dead patients')
plt.legend()
plt.subplot(212)
sns.distplot(essai2['age']/365, kde=False, label='patients alive')
plt.xlabel('age_at_diagnosis')
plt.legend()

PCA visualisation

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from Data_Loader import sort_data

x = StandardScaler().fit_transform(x)
pca = PCA(100)
pca.fit(x)
x = pca.transform(x)

In [ ]:
import matplotlib.pyplot as plt
principalDf = pd.DataFrame(data=x[:,0:2], columns = ['principal component 1', 'principal component 2'])
principalDf['yevent']= yevent
# finalDf = pd.concat([principalDf, yevent2], axis=1)

fig = plt.figure(figsize = (6,6))
ax = fig.add_subplot(1,1,1)
ax.set_xlabel('Principal Component 3', fontsize = 15)
ax.set_ylabel('Pincipal Component 4', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

targets = [0,1]
colors = ['r','g']
for target, color in zip(targets, colors): 
    indicesToKeep = principalDf['yevent']==target
    ax.scatter(principalDf.loc[indicesToKeep, 'principal component 1'], principalDf.loc[indicesToKeep, 'principal component 2'], c = color, s=50)
ax.legend(targets)
ax.grid()

Plot the Kaplan Meier curves

In [ ]:
from lifelines import KaplanMeierFitter

In [ ]:
import matplotlib.pyplot as plt 
kmf = KaplanMeierFitter()
kmf.fit(durations = ytime, event_observed = yevent)
plt.figure(figsize=(9,6))
kmf.plot_survival_function()
ax = plt.gca()




plt.xticks(fontsize = 12)
plt.yticks(fontsize=12)
plt.xlabel('Time [days]', fontfamily='times new roman', fontsize = 20)
plt.ylabel('Survival probability', fontfamily='times new roman', fontsize =20)
plt.legend(frameon=False, prop = {'family':'times new roman','size':17})
plt.title('Kaplan Meier curve: lung cancer',fontfamily='times new roman', fontsize = 20)



In [ ]:
import matplotlib.pyplot as plt 
kmf = KaplanMeierFitter()
kmf.fit(durations = ytime_p, event_observed = yevent_p)
plt.figure(figsize=(9,6))
kmf.plot_survival_function()
ax = plt.gca()




plt.xticks(fontsize = 12)
plt.yticks(fontsize=12)
plt.xlabel('Time [days]', fontfamily='times new roman', fontsize = 20)
plt.ylabel('Survival probability', fontfamily='times new roman', fontsize =20)
plt.legend(frameon=False, prop = {'family':'times new roman','size':17})
plt.title('Kaplan Meier curve: pancreas cancer',fontfamily='times new roman', fontsize = 20)

